# MCP

In [2]:
import os
import dotenv

from agents import Agent, Runner, trace, function_tool, WebSearchTool
from agents.mcp import MCPServerStreamableHttp

import chromadb
dotenv.load_dotenv()

True

In [3]:
# This is the same code as in the rag.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(
        query_texts=[query],
        n_results=max_results
    )
    
    if not results['documents'][0]:
        return f"No nutrition information found for: {query}"
    
    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][i]
        food_item = metadata['food_item'].title()
        calories = metadata['calories_per_100g']
        category = metadata['food_category'].title()
        
        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )
    
    return "Nutrition Information:\n" + "\n".join(formatted_results)


Let's integrate EXA Search as an MCP

In [4]:
exa_search_mcp = MCPServerStreamableHttp(
        name="Exa Search MCP",
        params={
            "url": f"https://mcp.exa.ai/mcp?{os.environ.get("EXA_API_KEY")}",
            "timeout": 30,
        },
        client_session_timeout_seconds=30,
        cache_tools_list=True,
        max_retry_attempts=1)

await exa_search_mcp.connect()

calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:  
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie 
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients. 
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times. 
    """,
    tools=[       
    calorie_lookup_tool],
    mcp_servers=[exa_search_mcp]
)


In [11]:
healthy_breakfast_planner_agent = Agent(
    name="Breakfast Planner Assistant",
    instructions="""
    * You are a helpful assistant that helps with healthy breakfast choices.
    * You give concise answers.
    Given the user's preferences prompt, come up with different breakfast meals that are healthy and fit for a busy person.
    * Explicitly mention the meal's names in your response along with a sentence of why this is a healthy choice.
    """
)


In [20]:
calorie_calculator_tool = calorie_agent_with_search.as_tool(tool_name="calorie-calculator", tool_description="Use this tool to calculate the calories of a meal and it's ingredients")
breakfast_planner_tool = healthy_breakfast_planner_agent.as_tool(tool_name="breakfast-planner", tool_description="Use this tool to plan a a number of healthy breakfast options")

breakfast_price_checker_agent = Agent(
    name="Breakfast Recommendation Assistant",
    instructions="""
    * You are a helpful assistant that takes multiple breakfast items (with ingredients and calories) and checks for the price of the ingredients.
    * Use the we search tool to get an approximate price for the ingredients. 
    * In your final output prove the meal name, ingredients with calories and price for each meal.
    * Use markdown and be as concise as possible.
    """,
    tools=[WebSearchTool()])

breakfast_advisor = Agent(
    name="Breakfast Advisor",
    instructions="""
    * You are a breakfast advisor. You come up with meal plans for the user based on their preferences. 
    * You also calculate the calories for the meal and its ingredients.
    * Based on the breakfast meals and the calories that you get from upstream agents, 
    * Create a meal plan for the user. For each meal, give a name, the ingredients, and the calories

    Follow this workflow carefully:
    1) Use the breakfast_planner_tool to plan a a number of healthy breakfast options.
    2) Use the calorie_calculator_tool to calculate the calories for the meal and its ingredients.
    3) Handoff the breakfast meals and the calories to the breakfast_price_checker_agent to check the price of the ingredients.

    """,
    tools=[breakfast_planner_tool, calorie_calculator_tool],
    handoff_description="""
    Create a concise breakfast recommendation based on the user's preferences and add the prices. Use Markdown format.
    """,
    handoffs=[breakfast_price_checker_agent],
    )


In [21]:
with trace("Multi-Agent Breakfast Advisor"):
    result = await Runner.run(breakfast_advisor, "I'm a busy person and I want to eat healthy breakfasts. I like to eat oatmeal and eggs. What is a healthy breakfast for me? Give me two options.")
    print(result.final_output)


Here are two healthy, quick options based on oatmeal and eggs.

1) Oatmeal-Egg Muffin Cups
- Ingredients (2 muffins): 1/2 cup rolled oats (40 g); 2 large eggs; 1/4 cup bell peppers; 1/4 cup spinach; 1 Tbsp shredded low-fat cheese; salt/pepper.
- Approx. calories per serving: ~271 kcal.
- Estimated price per meal (US, using typical store prices):
  - Oats: ~0.08 USD
  - Eggs (2): ~0.60 USD
  - Bell pepper: ~0.79 USD
  - Spinach: ~0.23 USD
  - Cheese (1 Tbsp): ~0.26 USD
  - Total: ~1.96 USD. Sources: oats key price (bulk 50 lb bag ~ $35.99, ≈ $0.72/lb), eggs (~$3.62 per dozen), spinach bag (~$2.28), bell pepper (~$0.79–$1.49 per pepper), shredded cheese (~$4.99–$5.23 per 7–8 oz). ([webstaurantstore.com](https://www.webstaurantstore.com/50-lb-bag-regular-rolled-oats/104NOA1150.html?utm_source=openai))

2) Savory Oatmeal with Egg Topping
- Ingredients: 1/2 cup rolled oats (40 g); 1 large egg; 1/2 cup sautéed mushrooms and spinach (water sauté); salt/pepper.
- Approx. calories per serving: 

Here are 4 healthy breakfast options centered on oatmeal and eggs, with approximate calories and core ingredients.

1) Savory Spinach & Mushroom Oatmeal Bowl with Fried Egg
- Core ingredients: 1/2 cup dry oats (42 g), 1 large egg fried in 1 tsp olive oil, 1 cup spinach, 1/4 cup mushrooms, salt, pepper
- Estimated calories: ~254 kcal

2) Sweet Berry Banana Overnight Oatmeal with Soft-Boiled Egg on the Side
- Core ingredients: 1/2 cup rolled oats, 1/2 cup milk or yogurt, 1/2 cup berries, 1 small banana, 1 hard-boiled egg
- Estimated calories: ~456 kcal

3) Oatmeal Egg Muffins (Savory Veggie Oat Muffins)
- Core ingredients: 1 cup rolled oats, 2 eggs, 1/2 cup diced vegetables (bell pepper, spinach), 1/4 cup shredded cheese
- Estimated calories: ~468 kcal per serving (recipe can be divided into multiple muffins)

4) Quick Oatmeal & Egg Scramble Jar
- Core ingredients: 1/2 cup oats, 2 eggs, 1/2 cup milk, 1/2 cup diced tomatoes, spices
- Estimated calories: ~435 kcal

Tips
- You can tailor ca